In [ ]:
import nltk; nltk.download('stopwords')
from nltk.util import ngrams

import re
import numpy as np
import pandas as pd
from pprint import pprint
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import LdaModel

import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from itertools import chain
import sys
import os
import pickle

from ast import literal_eval

from sklearn.feature_extraction.text import CountVectorizer
from nltk.probability import FreqDist

stop_words= stopwords.words('english')
stop_words.extend(['from','subject','re','edu','use'])

from gensim.test.utils import datapath
import utilities


#gensim 
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models import LdaModel

#spacy for lemmatisation
import spacy
#plot tools
import pyLDAvis
import pyLDAvis.gensim

from pprint import pprint


def sent_to_words(sentences):
        for sentence in sentences:
            yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts] 

def lemmatization(texts, allowed_postags=['NOUN','ADJ','VERB','ADV']):
    texts_out= []
    for sent in texts:
        doc= nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags]) 
    return texts_out

def cosine_similiarity(v1,v2):
        m1= np.linalg.norm(v1)
        m2= np.linalg.norm(v2)
        m= m1*m2
        d= np.dot(v1,v2)
        return d/(m)


#stores bag of words for comaprison
corpus_vectors=[]
v=[]

#list_files= sorted(os.listdir('../submitted_papers/'))
document_vectors=[]
row_lists=[]
load_dir="../neurips19/submitted_papers/"
data = []
doc_names = []   
token_len = 256

c=0
files_read=[]

#load data
base_folder='./data_info/loaded_pickles_nips19/'    
submitted_papers={}
with open(base_folder+'big_lemma_data_paper_submitted_nips19.pickle','rb' ) as ll:
    submitted_papers=pickle.load(ll)

data_submitted_papers=[]

for pid in submitted_papers:
    src= submitted_papers[pid]#['text']
    data_submitted_papers.append(src)

#prepare corpus for lda
corp=[]
for paper in data_submitted_papers:
    x= [p.split(' ') for p in paper]
    corp.append(x)
corp=[j for sub in corp for j in sub]

from gensim.corpora import Dictionary


id2word = corpora.Dictionary(corp)
id2word.filter_extremes(no_below=2, no_above=0.7)
corpus = [id2word.doc2bow(doc) for doc in corp] 
lda_model= gensim.models.ldamodel.LdaModel(corpus= corpus,  num_topics=25, random_state=100, update_every=1 , chunksize=100,passes=10, alpha='auto', per_word_topics= True)

pprint(lda_model.print_topics(num_topics=-1,num_words=12))


lda_model.save('./nips19_vectors2/model_lda_nips19')
id2word.save_as_text("./nips19_vectors2/wordlist_nips19_lda")